In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

from ch_3_metrics import *

In [2]:
advertising_data = pd.read_csv('data/advertising.csv', index_col=[0])

In [4]:
# regression on all predictors using sklearn - basic
X = advertising_data.iloc[:, :-1].to_numpy()
y = advertising_data.loc[:, 'sales'].to_numpy()

sk_regressor = LinearRegression()
sk_regressor.fit(X, y)
sk_regressor.coef_

array([ 0.04576465,  0.18853002, -0.00103749])

In [5]:
# regression on all predictors using statsmodels - detailed
X = sm.add_constant(X)
sm_regressor = sm.OLS(endog=y, exog=X)
results = sm_regressor.fit()

rse, r_squared, f_statistic = get_metrics(results)
print(results.summary().tables[1])
display_metrics(rse, r_squared, f_statistic)

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
Residual Standard Error: 1.69
R Squared: 0.9
F Statistic: 570.27

